[View in Colaboratory](https://colab.research.google.com/github/shraddharsh/colab/blob/master/tf_tryout.ipynb)

In [14]:
#import functions
from __future__ import print_function
import os, math
import pandas as pd
import numpy as np
from sklearn import metrics
from matplotlib import cm, gridspec, pyplot as plt
import tensorflow as tf
from tensorflow.python.data import Dataset

#Set options
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

#Loading dataset
california_housing_dataframe = pd.read_csv('https://storage.googleapis.com/mledu-datasets/california_housing_train.csv', sep=',')

# print(california_housing_dataframe)

california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe['median_house_value'] /= 1000
# california_housing_dataframe
# california_housing_dataframe.describe

my_feature = california_housing_dataframe[['total_rooms']]
feature_columns = [tf.feature_column.numeric_column('total_rooms')]

targets = california_housing_dataframe['median_house_value']

#Use Gradient Descent optimizer
my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

#Use Linear Regressor Model with a step of 0.0000001
linear_regressor = tf.estimator.LinearRegressor(feature_columns=feature_columns, optimizer=my_optimizer)

def my_input_fn( features, targets, batch_size=1, shuffle=True, num_epochs=None ):
  
  features = {key:np.array(value) for key, value in dict(features).items()}
  
  ds = Dataset.from_tensor_slices((features, targets))
  ds = ds.batch(batch_size).repeat(num_epochs)
  
  if shuffle:
    ds = ds.shuffle(buffer_size=10000)
  
  features, labels = ds.make_one_shot_iterator().get_next()
  return features, labels

_ = linear_regressor.train(
  input_fn = lambda:my_input_fn(my_feature, targets),
  steps=100
)


prediction_input_fn = lambda:my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)

predictions = linear_regressor.predict(input_fn=prediction_input_fn)

predictions = np.array([item['predictions'][0] for item in predictions])

mean_squared_error = metrics.mean_squared_error(predictions, targets)

root_mean_squared_error = math.sqrt(mean_squared_error)

print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

min_house_value = california_housing_dataframe["median_house_value"].min()
max_house_value = california_housing_dataframe["median_house_value"].max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)



Mean Squared Error (on training data): 56367.026
Root Mean Squared Error (on training data): 237.417
Min. Median House Value: 14.999
Max. Median House Value: 500.001
Difference between Min. and Max.: 485.002
Root Mean Squared Error: 237.417
